In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
# import plotly.io as pio
import re
# import rosbag_pandas
import math
import yaml
import pickle
import PyQt5
# from rosbag.bag import Bag

In [2]:
def failureProbability_linAcc(acc_z):
    sigma1 = 0.4
    sigma2 = -0.9
    mu = 0
    output = 1 - (1/(math.sqrt(2*math.pi)*sigma1))*(math.exp( - math.pow(acc_z - mu,2)/(2*math.pow(sigma2,2))))
    if(output>1):
        return 1
    elif(output<0):
        return 0
    else:
        return output

def magROC_cdf(x):
    a = -6
    b = -0.15
    try:
        p_f = 1/(1 + math.exp(-a*x + a*b))
    except OverflowError:
        p_f = float(0)
#         print('the values that caused this problem is:',x)
    return p_f


def odomError_cdf(timeSteps):
    k = 0.2
    p_f = k*timeSteps
    #This is a threshold to make the graphs clearer
    if(p_f>1):
        return 1
    elif(p_f<0):
        return 0
    else:
        return p_f

def vel_cdf(x):
    a = 1.5
    b = 2.5
    try:
        p_f = 1/(1 + math.exp(-a*x + a*b))
    except OverflowError:
        p_f = float(0)
    return p_f


def psnrError_cdf(x):
    a = 5
    b = 1
    try:
        p_f = 1/(1 + math.exp(-a*x + a*b))
    except OverflowError:
        p_f = float(0)
#         print('the values that caused this problem is:',x)
    return p_f

def probabilityOfFailure(arrayOfProbabilities):
    normalisationConstant = 1.0/(len(arrayOfProbabilities))
    p_f_total = 0.0
    for p_f in arrayOfProbabilities:
        p_f_total = p_f_total + p_f*normalisationConstant
    return p_f_total

In [3]:
def dataframe_parser(path_to_file, path_to_infoDict, arena, difficulty,trial):
#     print('Parsing Rosbag ',path_to_file)
#     df = rosbag_pandas.bag_to_dataframe(path_to_file)
#     info_dict = yaml.load(Bag(path_to_file, 'r')._get_yaml_info())
    # df.head()
    print('Parsing pickle file', path_to_file)
    df = pd.read_pickle(path_to_file)
    info_dict = pickle.load(open(path_to_infoDict, "rb"))
    timeToComplete = info_dict['duration']
    
    rv_columns = ['move_base_current_goal__pose_position_x',
                  'move_base_current_goal__pose_position_y',
                  'move_base_current_goal__pose_position_z',
                  'husky_velocity_controller_odom__pose_pose_position_x',
                  'husky_velocity_controller_odom__pose_pose_position_y',
                  'husky_velocity_controller_odom__pose_pose_position_z',
                  'odometry_filtered__pose_pose_position_x',
                  'odometry_filtered__pose_pose_position_y',
                  'odometry_filtered__pose_pose_position_z',
                  'imu_data__linear_acceleration_z',
                  'psnr_laserScan__data']

    df = df.fillna(method = 'ffill')
    df = df.fillna(0)
    df.describe()
    df['arena'] = arena
    df['difficulty'] = difficulty
    df['trial'] = trial
    
    df['move_base_current_goal__pose_position_x'] = 16.3375
    df['move_base_current_goal__pose_position_y']= 0.5000
    df['move_base_current_goal__pose_position_z']= 0.0000
    
    df['move_base_current_goal__pose_position_x'] = df['move_base_current_goal__pose_position_x'].apply(lambda x: np.float32(16.3375) if(x == 0) else None)
        
    df['odom_diff_x'] = df['husky_velocity_controller_odom__pose_pose_position_x'] - df['odometry_filtered__pose_pose_position_x']
    df['odom_diff_y'] = df['husky_velocity_controller_odom__pose_pose_position_y'] - df['odometry_filtered__pose_pose_position_y']
    df['odom_diff_z'] = df['husky_velocity_controller_odom__pose_pose_position_z'] - df['odometry_filtered__pose_pose_position_z']
    df['odom_posErr'] = np.linalg.norm(df[['odom_diff_x','odom_diff_y','odom_diff_z']].values, axis=1)

    df['magDist_diff_x'] = df['move_base_current_goal__pose_position_x'] - df['odometry_filtered__pose_pose_position_x']
    df['magDist_diff_y'] = df['move_base_current_goal__pose_position_y'] - df['odometry_filtered__pose_pose_position_y']
    df['magDist_diff_z'] = df['move_base_current_goal__pose_position_z'] - df['odometry_filtered__pose_pose_position_z']
    df['magDist_diff_x'] = df['magDist_diff_x'].astype('float')
    df['magDist_diff_y'] = df['magDist_diff_y'].astype('float')
    df['magDist_diff_z'] = df['magDist_diff_z'].astype('float')
    df['mag_distFromGoal'] = np.linalg.norm(df[['magDist_diff_x','magDist_diff_y','magDist_diff_z']].values, axis=1)
    
    df = df.reset_index()
    df = df.drop(columns = ['index'])
    samplingRate = 240
#     df.describe()
    df_sampled = pd.DataFrame(columns = ['imu_data__linear_acceleration_z','mag_distFromGoal','odom_posErr','psnr_laserScan__data','odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z','arena','difficulty','trial'])
    for i in range(0,len(df),samplingRate):
        df_sampled = df_sampled.append(df[['imu_data__linear_acceleration_z','mag_distFromGoal','odom_posErr','psnr_laserScan__data','odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z','arena','difficulty','trial']].iloc[i,:],ignore_index = True)
#     df_sampled = df_sampled.drop(index=df_sampled.index[:5],axis=0)
    
#     df_sampled.describe()

    df_sampled['mag_distFromGoal_roc'] = df_sampled['mag_distFromGoal'].diff(periods = 3).fillna(0)
    df_sampled['imu_data__linear_acceleration_z'].iloc[0] = df_sampled['imu_data__linear_acceleration_z'].iloc[1]
    # df_sampled['mag_distFromGoal_roc'] = df_sampled['mag_distFromGoal_roc'].fillna(0)
    df_sampled['imu_linAcc_z_roc'] = df_sampled['imu_data__linear_acceleration_z'].diff(2).fillna(method='bfill')
    df_sampled['odom_posErr_roc'] = df_sampled['odom_posErr'].diff().fillna(0)
#     df_sampled['psnr_laserScan__roc'] = df_sampled['psnr_laserScan__data'].diff(3).fillna(0)
#     df_sampled['psnr_laserScan__roc'].iloc[0] = df_sampled['psnr_laserScan__roc'].iloc[1]
    # df_sampled.head()
#     print('Came here')
    df_sampled['magROC_event'] = 500
    
    corr_vector = np.concatenate((np.linspace(0,-2.5,5), np.linspace(-2.5,0,6)[1:]), axis=None)
    for i in range(0,len(df_sampled['mag_distFromGoal_roc']) - len(corr_vector)):
        sum = 0
        for j in range(0,len(corr_vector)):
            sum = sum + (df_sampled['mag_distFromGoal_roc'].iloc[i+j] * corr_vector[len(corr_vector)-1 -j])
#         print('Im here bro')
        df_sampled['magROC_event'].iloc[i] = sum

#     corr_vector = np.linspace(-2.5,2.5,5)
#     for i in range(5,len(df_sampled['psnr_laserScan__roc']) - len(corr_vector)):
#         sum = 0
#         for j in range(0,len(corr_vector)):
#             sum = sum + (df_sampled['psnr_laserScan__roc'].iloc[i+j] * corr_vector[len(corr_vector)-1 -j])
# #         print('Im here bro')
#         df_sampled['psnr_event'].iloc[i] = sum    
   
    df_sampled['velocity_event'] = 0
    df_sampled['velocity_event_count'] = 0
    df_sampled['odom_mag'] = np.linalg.norm(df_sampled[['odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z']].values, axis=1)
    df_sampled['velocity_fromOdom'] = df_sampled['odom_mag'].diff(periods = 3)  
    df_sampled['velocity_event'] = df_sampled['velocity_fromOdom'].apply(lambda x: 1 if (x<0.01) else 0)
    a = df_sampled['velocity_event']!= 0
    df_sampled['velocity_event_count'] = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
    

#     i = 5
#     df_sampled['psnr_event'] = 0
#     while(i in range (5,len(df_sampled['psnr_laserScan__roc']) - 1)):
#         if(df_sampled['psnr_laserScan__roc'].iloc[i]<0.1):
#             count = 1
#             while(abs(df_sampled['psnr_laserScan__roc'].iloc[i+1] - df_sampled['psnr_laserScan__roc'].iloc[i])<0.1):
#                 df_sampled['psnr_event'].iloc[i] = count
#                 count = count + 1
#                 i = i+1
#         i = i+1
 
    df_sampled['odom_event'] = 0

    i = 1
    while(i in range(5,len(df_sampled['odom_posErr_roc']) - 1)):
    #     print('loop entered')
        if(((i+1)< len(df_sampled['odom_posErr_roc'])) and 
             (abs(df_sampled['odom_posErr_roc'].iloc[i]) < 0.001 and abs(df_sampled['odom_posErr_roc'].iloc[i+1]) < 0.001)
            ):
            count = 1
    #         print(i)
            while(((i+1)< len(df_sampled['odom_posErr_roc'])) and (abs(df_sampled['odom_posErr_roc'].iloc[i+1]) < 0.001)):
                df_sampled['odom_event'].iloc[i] = count
                count = count+1
                i = i+1
        i = i+1
    df_sampled['rv_magROC_event'] = df_sampled['magROC_event'].apply(magROC_cdf)
    df_sampled['rv_vel_event'] = df_sampled['velocity_event_count'].apply(vel_cdf)
    df_sampled['rv_odom_event'] = df_sampled['odom_event'].apply(odomError_cdf)
    df_sampled['rv_linAcc_Z_event'] = df_sampled['imu_linAcc_z_roc'].apply(failureProbability_linAcc)
    df_sampled['rv_snr_event'] = df_sampled['psnr_laserScan__data'].apply(psnrError_cdf)
#     df_sampled['p_f_total'] = df_sampled.apply(lambda x: probabilityOfFailure(np.array([x['rv_magROC_event'],x['rv_linAcc_Z_event'],x['rv_snr_event'],x['rv_vel_event']])),axis = 1)
    df_sampled['p_f_total'] = df_sampled.apply(lambda x: probabilityOfFailure(np.array([x['rv_magROC_event'],x['rv_odom_event'],x['rv_linAcc_Z_event'],x['rv_snr_event'],x['rv_vel_event']])),axis = 1)
    df_sampled['robot_health'] = (df_sampled['p_f_total']*np.log10(df_sampled['p_f_total'])).rolling(10).sum()
    df_sampled = df_sampled.reset_index()
    mean_health = df_sampled['robot_health'].mean()
    return [mean_health,timeToComplete,arena,difficulty,trial]

In [4]:
def dataframe_parser_realRobot(path_to_file, path_to_infoDict, arena, difficulty,trial):
    
    print('Parsing pickle file', path_to_file)
    df = pd.read_pickle(path_to_file)
    info_dict = pickle.load(open(path_to_infoDict, "rb"))
    timeToComplete = info_dict['duration']

    rv_columns = ['move_base_current_goal__pose_position_x',
                  'move_base_current_goal__pose_position_y',
                  'move_base_current_goal__pose_position_z',
                  'husky_velocity_controller_odom__pose_pose_position_x',
                  'husky_velocity_controller_odom__pose_pose_position_y',
                  'husky_velocity_controller_odom__pose_pose_position_z',
                  'odometry_filtered__pose_pose_position_x',
                  'odometry_filtered__pose_pose_position_y',
                  'odometry_filtered__pose_pose_position_z',
#                   'imu_data__linear_acceleration_z',
                  'psnr_laserScan__data']

    df = df.fillna(method = 'ffill')
    df = df.fillna(0)
    df.describe()
    df['arena'] = arena
    df['difficulty'] = difficulty
    df['trial'] = trial

#     df['move_base_current_goal__pose_position_x'] = 16.3375
#     df['move_base_current_goal__pose_position_y']= 0.5000
#     df['move_base_current_goal__pose_position_z']= 0.0000
    
#     df['move_base_current_goal__pose_position_x'] = df['move_base_current_goal__pose_position_x'].apply(lambda x: np.float32(16.3375) if(x == 0) else None)
        
    df['odom_diff_x'] = df['husky_velocity_controller_odom__pose_pose_position_x'] - df['odometry_filtered__pose_pose_position_x']
    df['odom_diff_y'] = df['husky_velocity_controller_odom__pose_pose_position_y'] - df['odometry_filtered__pose_pose_position_y']
    df['odom_diff_z'] = df['husky_velocity_controller_odom__pose_pose_position_z'] - df['odometry_filtered__pose_pose_position_z']
    df['odom_posErr'] = np.linalg.norm(df[['odom_diff_x','odom_diff_y','odom_diff_z']].values, axis=1)

    df['magDist_diff_x'] = df['move_base_current_goal__pose_position_x'] - df['odometry_filtered__pose_pose_position_x']
    df['magDist_diff_y'] = df['move_base_current_goal__pose_position_y'] - df['odometry_filtered__pose_pose_position_y']
    df['magDist_diff_z'] = df['move_base_current_goal__pose_position_z'] - df['odometry_filtered__pose_pose_position_z']
    df['magDist_diff_x'] = df['magDist_diff_x'].astype('float')
    df['magDist_diff_y'] = df['magDist_diff_y'].astype('float')
    df['magDist_diff_z'] = df['magDist_diff_z'].astype('float')
    df['mag_distFromGoal'] = np.linalg.norm(df[['magDist_diff_x','magDist_diff_y','magDist_diff_z']].values, axis=1)
    
    df = df.reset_index()
    df = df.drop(columns = ['index'])
    samplingRate = 240
#     df.describe()
    df_sampled = pd.DataFrame(columns = ['mag_distFromGoal','odom_posErr','psnr_laserScan__data','odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z','arena','difficulty','trial'])
    for i in range(0,len(df),samplingRate):
        df_sampled = df_sampled.append(df[['mag_distFromGoal','odom_posErr','psnr_laserScan__data','odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z','arena','difficulty','trial']].iloc[i,:],ignore_index = True)
#     df_sampled = df_sampled.drop(index=df_sampled.index[:5],axis=0)
    
#     df_sampled.describe()

    df_sampled['mag_distFromGoal_roc'] = df_sampled['mag_distFromGoal'].diff(periods = 3).fillna(0)
#     df_sampled['imu_data__linear_acceleration_z'].iloc[0] = df_sampled['imu_data__linear_acceleration_z'].iloc[1]
    # df_sampled['mag_distFromGoal_roc'] = df_sampled['mag_distFromGoal_roc'].fillna(0)
#     df_sampled['imu_linAcc_z_roc'] = df_sampled['imu_data__linear_acceleration_z'].diff(2).fillna(method='bfill')
    df_sampled['odom_posErr_roc'] = df_sampled['odom_posErr'].diff().fillna(0)
#     df_sampled['psnr_laserScan__roc'] = df_sampled['psnr_laserScan__data'].diff(3).fillna(0)
#     df_sampled['psnr_laserScan__roc'].iloc[0] = df_sampled['psnr_laserScan__roc'].iloc[1]
    # df_sampled.head()
#     print('Came here')
    df_sampled['magROC_event'] = 500
    
    corr_vector = np.concatenate((np.linspace(0,-2.5,5), np.linspace(-2.5,0,6)[1:]), axis=None)
    for i in range(0,len(df_sampled['mag_distFromGoal_roc']) - len(corr_vector)):
        sum = 0
        for j in range(0,len(corr_vector)):
            sum = sum + (df_sampled['mag_distFromGoal_roc'].iloc[i+j] * corr_vector[len(corr_vector)-1 -j])
#         print('Im here bro')
        df_sampled['magROC_event'].iloc[i] = sum

#     corr_vector = np.linspace(-2.5,2.5,5)
#     for i in range(5,len(df_sampled['psnr_laserScan__roc']) - len(corr_vector)):
#         sum = 0
#         for j in range(0,len(corr_vector)):
#             sum = sum + (df_sampled['psnr_laserScan__roc'].iloc[i+j] * corr_vector[len(corr_vector)-1 -j])
# #         print('Im here bro')
#         df_sampled['psnr_event'].iloc[i] = sum    
   
    df_sampled['velocity_event'] = 0
    df_sampled['velocity_event_count'] = 0
    df_sampled['odom_mag'] = np.linalg.norm(df_sampled[['odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z']].values, axis=1)
    df_sampled['velocity_fromOdom'] = df_sampled['odom_mag'].diff(periods = 3)  
    df_sampled['velocity_event'] = df_sampled['velocity_fromOdom'].apply(lambda x: 1 if (x<0.01) else 0)
    a = df_sampled['velocity_event']!= 0
    df_sampled['velocity_event_count'] = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
    

#     i = 5
#     df_sampled['psnr_event'] = 0
#     while(i in range (5,len(df_sampled['psnr_laserScan__roc']) - 1)):
#         if(df_sampled['psnr_laserScan__roc'].iloc[i]<0.1):
#             count = 1
#             while(abs(df_sampled['psnr_laserScan__roc'].iloc[i+1] - df_sampled['psnr_laserScan__roc'].iloc[i])<0.1):
#                 df_sampled['psnr_event'].iloc[i] = count
#                 count = count + 1
#                 i = i+1
#         i = i+1
 
    df_sampled['odom_event'] = 0

    i = 1
    while(i in range(5,len(df_sampled['odom_posErr_roc']) - 1)):
    #     print('loop entered')
        if(((i+1)< len(df_sampled['odom_posErr_roc'])) and 
             (abs(df_sampled['odom_posErr_roc'].iloc[i]) < 0.001 and abs(df_sampled['odom_posErr_roc'].iloc[i+1]) < 0.001)
            ):
            count = 1
    #         print(i)
            while(((i+1)< len(df_sampled['odom_posErr_roc'])) and (abs(df_sampled['odom_posErr_roc'].iloc[i+1]) < 0.001)):
                df_sampled['odom_event'].iloc[i] = count
                count = count+1
                i = i+1
        i = i+1
    df_sampled['rv_magROC_event'] = df_sampled['magROC_event'].apply(magROC_cdf)
    df_sampled['rv_vel_event'] = df_sampled['velocity_event_count'].apply(vel_cdf)
    df_sampled['rv_odom_event'] = df_sampled['odom_event'].apply(odomError_cdf)
#     df_sampled['rv_linAcc_Z_event'] = df_sampled['imu_linAcc_z_roc'].apply(failureProbability_linAcc)
    df_sampled['rv_snr_event'] = df_sampled['psnr_laserScan__data'].apply(psnrError_cdf)
#     df_sampled['p_f_total'] = df_sampled.apply(lambda x: probabilityOfFailure(np.array([x['rv_magROC_event'],x['rv_linAcc_Z_event'],x['rv_snr_event'],x['rv_vel_event']])),axis = 1)
    df_sampled['p_f_total'] = df_sampled.apply(lambda x: probabilityOfFailure(np.array([x['rv_magROC_event'],x['rv_odom_event'],x['rv_snr_event'],x['rv_vel_event']])),axis = 1)
    df_sampled['robot_health'] = (df_sampled['p_f_total']*np.log10(df_sampled['p_f_total'])).rolling(10).sum()
    df_sampled = df_sampled.reset_index()
    mean_health = df_sampled['robot_health'].mean()
    return [mean_health,timeToComplete,arena,difficulty,trial]

In [5]:
import os
directory = r'/Volumes/Aniketh/rosbags/RealRobotExp_Combined'
# n = len(os.listdir(directory))
matrix = []
# count = 0
for filename in os.listdir(directory):
    if filename.endswith('bag'):
        df_filename = filename[:-3]+'pickle'
        info_dict_filename = filename[:-4]+'__info_dict.pickle'
        myList = dataframe_parser_realRobot(os.path.join(directory, df_filename),os.path.join(directory, info_dict_filename),filename.split('_')[0],filename.split('_')[1],filename.split('_')[2][:-4])
        matrix.append(myList)
#         matrix[count,1] = myArray[1]
#         count = count + 1

df = pd.DataFrame(matrix,columns = ['average_robotHealth','timeTaken','Condition','Difficulty','Trial'])


Parsing pickle file /Volumes/Aniketh/rosbags/RealRobotExp_Combined/real_obstacles_8.pickle


/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Parsing pickle file /Volumes/Aniketh/rosbags/RealRobotExp_Combined/real_obstacles_9.pickle


/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Parsing pickle file /Volumes/Aniketh/rosbags/RealRobotExp_Combined/real_obstacles_10.pickle


...

/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Parsing pickle file /Volumes/Aniketh/rosbags/RealRobotExp_Combined/real_roughTerrainLaserNoise_06.pickle


/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Parsing pickle file /Volumes/Aniketh/rosbags/RealRobotExp_Combined/real_roughTerrainLaserNoise_07.pickle


/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
df['Difficulty'] = df['Difficulty'].replace('obstacles','Baseline')
df['Difficulty'] = df['Difficulty'].replace('roughTerrain','Rough Terrain')
df['Difficulty'] = df['Difficulty'].replace('roughTerrainLaserNoise','Noise and Rough Terrain')

In [8]:
df

,average_robotHealth,timeTaken,Condition,Difficulty,Trial,Level
0,-0.960648,53.780558,real,Baseline,8,NaN
1,-0.755479,46.443009,real,Baseline,9,NaN
2,-1.017322,48.263377,real,Baseline,10,NaN
3,-1.004726,55.283123,real,Baseline,11,NaN
4,-0.821814,48.287068,real,Baseline,12,NaN
5,-0.789254,42.748224,real,Baseline,13,NaN
6,-0.882956,46.686962,real,Baseline,14,NaN
7,-1.476356,100.008975,real,Rough Terrain,8,NaN
8,-1.305236,104.861324,real,Rough Terrain,14,NaN
9,-1.216439,55.832678,real,Rough Terrain,13,NaN


In [9]:
df.loc[(df.Difficulty == 'Baseline'), 'Level'] = 'Level 1'
df.loc[(df.Difficulty == 'Rough Terrain'), 'Level'] = 'Level 2'
df.loc[(df.Difficulty == 'Noise and Rough Terrain'), 'Level'] = 'Level 3'

In [10]:
df_levels = df.loc[df['Level'] != '']
df_levels['Condition'] = df_levels['Difficulty']
df_levels

,average_robotHealth,timeTaken,Condition,Difficulty,Trial,Level
0,-0.960648,53.780558,Baseline,Baseline,8,Level 1
1,-0.755479,46.443009,Baseline,Baseline,9,Level 1
2,-1.017322,48.263377,Baseline,Baseline,10,Level 1
3,-1.004726,55.283123,Baseline,Baseline,11,Level 1
4,-0.821814,48.287068,Baseline,Baseline,12,Level 1
5,-0.789254,42.748224,Baseline,Baseline,13,Level 1
6,-0.882956,46.686962,Baseline,Baseline,14,Level 1
7,-1.476356,100.008975,Rough Terrain,Rough Terrain,8,Level 2
8,-1.305236,104.861324,Rough Terrain,Rough Terrain,14,Level 2
9,-1.216439,55.832678,Rough Terrain,Rough Terrain,13,Level 2


In [11]:
%matplotlib qt
ax =sns.scatterplot(data=df_levels, x="timeTaken", y="average_robotHealth",hue='Condition')
ax.set_title('Correlation Plot for Exp II. Spearman\'s Rho Value = -0.76 ')
ax.set(ylabel = 'Average Robot Health over run time', xlabel = 'Time taken to complete the task')
# ax.legend(title='Condition',loc='upper right')

[Text(0, 0.5, 'Average Robot Health over run time'),
 Text(0.5, 0, 'Time taken to complete the task')]

In [12]:
corr_matrix = df_levels[['timeTaken','average_robotHealth']].corr( method = 'pearson')

In [13]:
corr_matrix

,timeTaken,average_robotHealth
timeTaken,1.000000,-0.781139
average_robotHealth,-0.781139,1.000000


In [14]:
!python --version

Python 3.6.9 :: Anaconda, Inc.


In [15]:
%matplotlib qt
ax = sns.boxplot(x="Level", y="timeTaken",hue="Condition", data=df_levels)
ax.set_title('Time taken vs performance degradation in Exp II')
ax.set(xlabel = 'Levels of Performance degradation', ylabel = 'Time taken to complete the task')
# ax.legend(title='Condition',loc='upper left')

[Text(0.5, 0, 'Levels of Performance degradation'),
 Text(0, 0.5, 'Time taken to complete the task')]

In [16]:
%matplotlib qt
ax = sns.boxplot(x="Level", y="average_robotHealth",hue="Condition", data=df_levels)
ax.set_title('Run Time Average Robot Health vs performance degradation in Exp II')
ax.set(xlabel = 'Levels of Performance degradation', ylabel = 'Average Robot Health over run time')
# ax.legend(title='Condition',loc='upper left')

[Text(0.5, 0, 'Levels of Performance degradation'),
 Text(0, 0.5, 'Average Robot Health over run time')]

In [24]:
%matplotlib qt

fig, axs = plt.subplots(1,2)
sns.boxplot(x="Level", y="timeTaken",hue="Condition", data=df_levels, ax = axs[0])
# axs[0].set_title('Time taken vs performance degradation in Exp II')
# axs[0].set(xlabel = 'Levels of Performance degradation', ylabel = 'Time taken to complete the task (in seconds)')

sns.boxplot(x="Level", y="average_robotHealth",hue="Condition", data=df_levels, ax = axs[1])
# axs[1].set_title('Average Robot Health vs performance degradation in Exp II')
# axs[1].set(xlabel = 'Levels of Performance degradation', ylabel = 'Average Robot Health over run time')



axs[0].set_title('Time taken vs performance degradation in Exp II', fontsize='large')
axs[0].set_xlabel('Levels of Performance degradation',fontsize='large')
axs[0].set_ylabel('Time taken to complete the task (in seconds)',fontsize='large')
axs[0].legend(fontsize=14)

# sns.boxplot(x="Level", y="average_robotHealth",hue="Condition", data=df_levels, ax = axs[1])
axs[1].set_title('Run Time Average Robot Health vs performance degradation in Exp II',fontsize='large')
axs[1].set_xlabel ('Levels of Performance degradation',fontsize='large')
axs[1].set_ylabel ('Average Robot Health over run time',fontsize='large')
axs[1].legend(fontsize=14)

In [37]:
df_levels['timeTaken'].values

array([ 53.780558,  46.443009,  48.263377,  55.283123,  48.287068,
        42.748224,  46.686962, 100.008975, 104.861324,  55.832678,
        54.68893 ,  58.820584, 104.996879,  52.590738,  74.905375,
        92.446682,  81.617253,  56.558838,  61.084594,  62.26261 ,
        70.768572,  53.780558,  46.443009,  48.263377,  55.283123,
        48.287068,  42.748224,  46.686962,  58.010998,  57.233429,
        53.164307,  70.129969,  60.968902,  55.302336,  50.145012,
        65.541155, 100.85954 ,  98.503147,  86.006693,  62.690715,
        61.069231, 112.353158])